In [1]:
import numpy as np
import matplotlib.pyplot as plt
import camb
from camb import model, initialpower
import andreap
from scipy.interpolate import interp1d

import importlib.util
import sys

In [2]:
''' This messiness is necessary to import scripts which are not installed and
which are not located in the cwd.'''

spec = importlib.util.spec_from_file_location(
    "camb_interface", "../../nuCOMET/camb_interface.py")
ci = importlib.util.module_from_spec(spec)
sys.modules["camb_interface"] = ci
spec.loader.exec_module(ci)

spec = importlib.util.spec_from_file_location(
    "generate_training_data", "../../nuCOMET/generate_training_data.py")
gtd = importlib.util.module_from_spec(spec)
sys.modules["generate_training_data"] = gtd
spec.loader.exec_module(gtd)

In [3]:
hc_massive_pred = np.load("hc_backup_i4999_massive_pred_revised.npy",
    allow_pickle=True)
samples_massive_pred = np.load("samples_backup_i4999_massive_pred_revised.npy",
    allow_pickle=True)
standard_k = np.load("standard_k.npy", allow_pickle=True)

In [4]:
first_cosmology = hc_massive_pred[0]
first_pspec = samples_massive_pred[0]

In [5]:
ombh2 = first_cosmology[0]
omch2 = first_cosmology[1]
ns = first_cosmology[2]
sigma12 = first_cosmology[3]
As = first_cosmology[4]
omnuh2 = first_cosmology[5]

In [6]:
full_cosm = gtd.build_cosmology(ombh2, omch2, ns, sigma12, As, omnuh2)

In [7]:
mnu = full_cosm['mnu']

In [19]:
Pk_lo_h = andreap.get_PK(ombh2, omch2, ns, mnu, 10, As)
Pk_hi_h = andreap.get_PK(ombh2, omch2, ns, mnu, 67, As)
Pk_target_h = andreap.get_PK(ombh2, omch2, ns, mnu, 57, As)

0.008207
1
0.008207
1
0.008207
1


In [9]:
z = np.linspace(0., 5., 500)
sig12 = np.zeros(500)
for i in range(500):
    sig12[i] = andreap.get_s12(Pk['mzero'], z[i])

In [10]:
sigma12

0.9719207949661621

In [11]:
sig12[0]

1.3210696340492791

In [12]:
sig12_interpolator = interp1d(sig12, z, kind='cubic')

In [13]:
g_pspec, g_sigma12, g_z = gtd.psz(full_cosm, standard_k)

Discrepancy between maximal achievable sigma12 and target -0.03799240966654416
Desired 0.9719207949661621
Discrepancy between maximal achievable sigma12 and target 0.03209088418396622
Desired 0.9719207949661621
We had to move h to 0.57


In [15]:
import copy as cp
tilde_cosm = cp.deepcopy(full_cosm)
tilde_cosm['omch2'] += tilde_cosm['omnuh2']
tilde_cosm = ci.specify_neutrino_mass(tilde_cosm, 0, 0)

In [16]:
kci, zci, pci, sci = ci.kzps(tilde_cosm)

In [17]:
sci

array([1.00401168])

In [20]:
andreap.get_s12(Pk_hi_h['mzero'], 0)

0.7470678684364096

In [24]:
andreap.get_s12_fixedz(Pk_hi_h['mzero'], 0)

TypeError: __call__() missing 1 required positional argument: 'y'

In [21]:
tilde_cosm

Name           Columbus_0
ombh2             0.02308
omch2            0.117574
n_s              0.946505
A_s                   0.0
OmB                  0.05
OmC              0.268584
OmM              0.318584
OmK                   0.0
OmL              0.681416
h                    0.57
w0                   -1.0
wa                   0.00
EOmDE                   -
z(4)                  2.0
z(3)                  1.0
z(2)                 0.57
z(1)                  0.3
z(0)                  0.0
Lbox               1000.0
sigma8            0.82755
sigma12          0.971921
omnuh2                  0
mnu                   0.0
nnu_massive             0
Name: 0, dtype: object